In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [2]:
import json

from pprint import pprint

In [3]:
val_df = pd.read_json('data/val.json', orient = 'index')
val_df.reset_index(inplace=True)
val_df = val_df.rename(columns = {'index': 'user_id'})

In [4]:
val_df

,user_id,target,features
0,user_127756,female,"{'orders': [{'site-id': 407, 'orders': [{'crea..."
1,user_127757,male,"{'orders': [{'site-id': 16, 'orders': [{'creat..."
2,user_127758,female,"{'orders': [{'site-id': 149, 'orders': [{'crea..."
3,user_127759,female,"{'orders': [{'site-id': 93, 'orders': [{'creat..."
4,user_127760,female,"{'visits': [{'site-id': 3, 'first-seen': 16836..."
...,...,...,...
27442,user_155198,female,"{'orders': [{'site-id': 7, 'orders': [{'create..."
27443,user_155199,male,"{'orders': [{'site-id': 21, 'orders': [{'creat..."
27444,user_155200,female,"{'orders': [{'site-id': 34, 'orders': [{'creat..."
27445,user_155201,male,"{'orders': [{'site-id': 218, 'orders': [{'crea..."


In [5]:
val_df_fixed = val_df.join(pd.json_normalize(val_df['features'])).drop('features', axis='columns')
val_df_fixed

,user_id,target,orders,visits,site-meta,exchange-sessions,last-visits-in-categories
0,user_127756,female,"[{'site-id': 407, 'orders': [{'created-at': 16...","[{'site-id': 3, 'first-seen': 1697125590, 'las...",[{'site-id': 407}],"[{'landed-at': 1697132982, 'sites': [169, 214,...","[{'category': 'other', 'last-visit-at': 169713..."
1,user_127757,male,"[{'site-id': 16, 'orders': [{'created-at': 163...","[{'site-id': 3, 'first-seen': 1698423484, 'las...","[{'site-id': 16, 'recency': 1, 'frequency': 5,...","[{'landed-at': 1693508972, 'sites': [2, 169, 7...","[{'category': 'electronics', 'last-visit-at': ..."
2,user_127758,female,"[{'site-id': 149, 'orders': [{'created-at': 16...","[{'site-id': 391, 'first-seen': 1685615371, 'l...","[{'site-id': 391}, {'site-id': 42}, {'site-id'...",NaN,"[{'category': 'sport', 'last-visit-at': 165060..."
3,user_127759,female,"[{'site-id': 93, 'orders': [{'created-at': 162...","[{'site-id': 495, 'first-seen': 1687325784, 'l...","[{'site-id': 93, 'recency': 5, 'frequency': 2,...","[{'landed-at': 1697791749, 'sites': [373, 169,...","[{'category': 'furniture', 'last-visit-at': 16..."
4,user_127760,female,NaN,"[{'site-id': 3, 'first-seen': 1683626530, 'las...","[{'site-id': 46}, {'site-id': 29}, {'site-id':...",NaN,"[{'category': 'other', 'last-visit-at': 169765..."
...,...,...,...,...,...,...,...
27442,user_155198,female,"[{'site-id': 7, 'orders': [{'created-at': 1625...","[{'site-id': 3, 'first-seen': 1688903751, 'las...","[{'site-id': 49, 'recency': 1, 'frequency': 1,...","[{'landed-at': 1650900543, 'sites': [71, 326, ...","[{'category': 'other', 'last-visit-at': 169699..."
27443,user_155199,male,"[{'site-id': 21, 'orders': [{'created-at': 169...","[{'site-id': 21, 'first-seen': 1697887543, 'la...","[{'site-id': 21}, {'site-id': 17}]",NaN,"[{'category': 'hypermarket', 'last-visit-at': ..."
27444,user_155200,female,"[{'site-id': 34, 'orders': [{'created-at': 158...","[{'site-id': 22, 'first-seen': 1686412090, 'la...","[{'site-id': 34, 'recency': 1, 'frequency': 1,...",NaN,"[{'category': 'furniture', 'last-visit-at': 16..."
27445,user_155201,male,"[{'site-id': 218, 'orders': [{'created-at': 16...","[{'site-id': 555, 'first-seen': 1696063525, 'l...","[{'site-id': 218, 'recency': 1, 'frequency': 1...",NaN,"[{'category': 'other', 'last-visit-at': 169911..."


In [76]:
def explode_column(df, level_labels):
    for level in level_labels:
        df = df.explode(level, ignore_index=True)
        df = df.rename(columns = {level: level+'_old'})
        df = df.join(pd.json_normalize(df[level+'_old'])).drop(level+'_old', axis='columns')
    return df

# orders

In [7]:
val_orders = val_df_fixed[['user_id', 'target', 'orders']].copy()
val_orders

,user_id,target,orders
0,user_127756,female,"[{'site-id': 407, 'orders': [{'created-at': 16..."
1,user_127757,male,"[{'site-id': 16, 'orders': [{'created-at': 163..."
2,user_127758,female,"[{'site-id': 149, 'orders': [{'created-at': 16..."
3,user_127759,female,"[{'site-id': 93, 'orders': [{'created-at': 162..."
4,user_127760,female,NaN
...,...,...,...
27442,user_155198,female,"[{'site-id': 7, 'orders': [{'created-at': 1625..."
27443,user_155199,male,"[{'site-id': 21, 'orders': [{'created-at': 169..."
27444,user_155200,female,"[{'site-id': 34, 'orders': [{'created-at': 158..."
27445,user_155201,male,"[{'site-id': 218, 'orders': [{'created-at': 16..."


In [77]:
levels_labels = ['orders', 'orders', 'items']
val_orders_flat = explode_column(val_orders, levels_labels)
val_orders_flat

,user_id,target,site-id,created-at,id,count,general-category-path,brand-id
0,user_127756,female,407.0,1.697126e+09,item_676374,NaN,NaN,NaN
1,user_127757,male,16.0,1.636574e+09,item_3780263,2.0,"[90578, 90574, 198118]",1237.0
2,user_127757,male,16.0,1.637513e+09,item_5403882,2.0,"[90578, 90574, 198118]",1237.0
3,user_127757,male,16.0,1.637948e+09,item_1627033,1.0,"[91498, 91497, 91461, 198119]",1.0
4,user_127757,male,16.0,1.638631e+09,item_554781,1.0,"[90629, 90625, 10599873, 198119]",3519.0
...,...,...,...,...,...,...,...,...
815454,user_155202,female,53.0,1.684604e+09,item_650360,1.0,NaN,NaN
815455,user_155202,female,53.0,1.684604e+09,item_650361,1.0,NaN,NaN
815456,user_155202,female,54.0,1.420435e+09,NaN,NaN,NaN,NaN
815457,user_155202,female,300.0,1.688068e+09,item_650362,1.0,"[818863, 15068776, 7811881, 7877999]",915491.0


In [80]:
def get_features_from_orders(x):
    features = {}
    features['site-id_count'] = x['site-id'].count()
    features['site-id_unique'] = x['site-id'].nunique()
    features['site-id_mode'] = x['site-id'].mode().tolist()#.to_string(index=False)
    features['id_count'] = x['id'].count()
    features['id_count_unique'] = x['id'].nunique()
    features['id_mode'] = x['id'].mode().tolist()#.to_string(index=False)
    features['count_sum'] = x['count'].sum()
    features['brand-id_mode'] = x['brand-id'].mode().tolist()#.to_string(index=False) 
    features['brand-id_unique'] = x['brand-id'].nunique()
    return pd.Series(features, index=['site-id_count',
                                      'site-id_unique',
                                      'site-id_mode',
                                      'id_count',
                                      'id_count_unique',
                                      'id_mode',
                                      'count_sum',
                                      'brand-id_mode',
                                      'brand-id_unique'])

In [81]:
val_orders_grouped = val_orders_flat.groupby(['user_id', 'target']).apply(get_features_from_orders)
val_orders_grouped = val_orders_grouped.reset_index()
val_orders_grouped = val_orders_grouped.replace('Series([], )', np.nan)
val_orders_grouped

,user_id,target,site-id_count,site-id_unique,site-id_mode,id_count,id_count_unique,id_mode,count_sum,brand-id_mode,brand-id_unique
0,user_127756,female,1,1,[407.0],1,1,[item_676374],0.0,[],0
1,user_127757,male,16,4,[16.0],14,14,"[item_1399130, item_1627033, item_1898950, ite...",16.0,"[1055.0, 1237.0]",6
2,user_127758,female,38,5,[21.0],38,35,"[item_2072631, item_3362928, item_611359]",67.0,[15820.0],21
3,user_127759,female,52,3,[93.0],52,46,"[item_47281, item_47282]",52.0,[3192.0],3
4,user_127760,female,0,0,[],0,0,[],0.0,[],0
...,...,...,...,...,...,...,...,...,...,...,...
27442,user_155198,female,9,6,"[7.0, 8.0, 49.0]",7,6,[item_51490],9.0,[1889758.0],3
27443,user_155199,male,3,1,[21.0],0,0,[],0.0,[],0
27444,user_155200,female,2,1,[34.0],2,2,"[item_1648, item_6173373]",2.0,[708.0],1
27445,user_155201,male,3,2,[218.0],0,0,[],0.0,[],0


In [95]:
sites_stat = val_orders_flat[['user_id', 'target', 'site-id']]
sites_stat = sites_stat.drop_duplicates()
sites_stat

,user_id,target,site-id
0,user_127756,female,407.0
1,user_127757,male,16.0
11,user_127757,male,40.0
12,user_127757,male,288.0
15,user_127757,male,21.0
...,...,...,...
815438,user_155202,female,97.0
815446,user_155202,female,53.0
815456,user_155202,female,54.0
815457,user_155202,female,300.0


In [99]:
sites_stat.groupby(['target'])['site-id'].agg(pd.Series.mode)

target
female    21.0
male       2.0
Name: site-id, dtype: float64

In [133]:
top20_sites_female = sites_stat[sites_stat['target']=='female']['site-id'].value_counts().nlargest(20)
top20_sites_female

site-id
21.0     3191
8.0      3029
2.0      3027
20.0     2352
29.0     2149
179.0    2120
49.0     1950
34.0     1857
62.0     1805
16.0     1752
54.0     1463
40.0     1277
39.0     1223
7.0      1134
50.0     1116
149.0    1082
98.0     1022
119.0     993
46.0      992
147.0     944
Name: count, dtype: int64

In [131]:
top20_sites_male = sites_stat[sites_stat['target']=='male']['site-id'].value_counts().nlargest(20)
top20_sites_male

site-id
2.0      3812
21.0     3033
20.0     2520
29.0     2159
16.0     1891
40.0     1684
49.0     1502
147.0    1382
39.0     1279
8.0      1263
30.0     1144
62.0     1105
54.0      985
151.0     983
179.0     950
489.0     897
193.0     730
119.0     715
7.0       695
441.0     691
Name: count, dtype: int64

In [121]:
if 3.0 in sites_stat['site-id'].value_counts().index.tolist():
    print('YES')
else:
    print('NO')

NO


In [134]:
set(top20_sites_female.index.tolist()) == set(top20_sites_male.index.tolist())

False

In [135]:
set(top20_sites_female.index.tolist()) - set(top20_sites_male.index.tolist())

{34.0, 46.0, 50.0, 98.0, 149.0}

In [136]:
set(top20_sites_male.index.tolist()) - set(top20_sites_female.index.tolist())

{30.0, 151.0, 193.0, 441.0, 489.0}

In [82]:
val_orders_grouped['site-id_unique'].value_counts()

site-id_unique
1      5209
2      4271
3      3192
0      2576
4      2421
       ... 
122       1
100       1
124       1
81        1
133       1
Name: count, Length: 125, dtype: int64

In [83]:
val_orders_grouped['site-id_mode'].value_counts()

site-id_mode
[]               2576
[2.0]            2264
[20.0]           1587
[8.0]            1217
[21.0]            842
                 ... 
[16.0, 35.0]        1
[8.0, 371.0]        1
[1067.0]            1
[13.0, 40.0]        1
[53.0, 135.0]       1
Name: count, Length: 2952, dtype: int64

In [84]:
val_orders_grouped['site-id_count'].value_counts()

site-id_count
1      2822
0      2576
2      2355
3      1737
4      1385
       ... 
395       1
509       1
573       1
595       1
722       1
Name: count, Length: 568, dtype: int64

# visits

In [85]:
val_visits = val_df_fixed[['user_id', 'target', 'visits']].copy()
val_visits

,user_id,target,visits
0,user_127756,female,"[{'site-id': 3, 'first-seen': 1697125590, 'las..."
1,user_127757,male,"[{'site-id': 3, 'first-seen': 1698423484, 'las..."
2,user_127758,female,"[{'site-id': 391, 'first-seen': 1685615371, 'l..."
3,user_127759,female,"[{'site-id': 495, 'first-seen': 1687325784, 'l..."
4,user_127760,female,"[{'site-id': 3, 'first-seen': 1683626530, 'las..."
...,...,...,...
27442,user_155198,female,"[{'site-id': 3, 'first-seen': 1688903751, 'las..."
27443,user_155199,male,"[{'site-id': 21, 'first-seen': 1697887543, 'la..."
27444,user_155200,female,"[{'site-id': 22, 'first-seen': 1686412090, 'la..."
27445,user_155201,male,"[{'site-id': 555, 'first-seen': 1696063525, 'l..."


In [86]:
levels_labels = ['visits', 'visits']
val_visits_flat = explode_column(val_visits, levels_labels)
val_visits_flat

,user_id,target,site-id,first-seen,last-seen,visited-at,session-duration,pages-count,visited-items,visited-general-categories,visited-universal-brands
0,user_127756,female,3.0,1.697126e+09,1.697133e+09,1.697126e+09,0.0,1.0,NaN,NaN,NaN
1,user_127756,female,3.0,1.697126e+09,1.697133e+09,1.697133e+09,0.0,1.0,NaN,NaN,NaN
2,user_127756,female,407.0,1.697125e+09,1.699115e+09,1.698855e+09,626.0,3.0,NaN,NaN,NaN
3,user_127756,female,407.0,1.697125e+09,1.699115e+09,1.698931e+09,0.0,1.0,NaN,NaN,NaN
4,user_127756,female,407.0,1.697125e+09,1.699115e+09,1.698947e+09,0.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
561073,user_155201,male,13.0,1.691565e+09,1.697718e+09,1.697236e+09,581.0,3.0,NaN,NaN,NaN
561074,user_155201,male,13.0,1.691565e+09,1.697718e+09,1.697718e+09,0.0,1.0,NaN,NaN,NaN
561075,user_155202,female,3.0,1.696679e+09,1.696864e+09,1.696679e+09,0.0,1.0,NaN,NaN,NaN
561076,user_155202,female,3.0,1.696679e+09,1.696864e+09,1.696680e+09,0.0,1.0,NaN,NaN,NaN


In [110]:
sites_stat_visits = val_visits_flat[['user_id', 'target', 'site-id']]
sites_stat_visits = sites_stat_visits.drop_duplicates()
sites_stat_visits

,user_id,target,site-id
0,user_127756,female,3.0
2,user_127756,female,407.0
12,user_127757,male,3.0
13,user_127757,male,308.0
14,user_127757,male,16.0
...,...,...,...
561063,user_155201,male,3.0
561067,user_155201,male,10.0
561068,user_155201,male,209.0
561069,user_155201,male,13.0


In [111]:
sites_stat_visits.groupby(['target'])['site-id'].agg(pd.Series.mode)

target
female    3.0
male      3.0
Name: site-id, dtype: float64

In [112]:
sites_stat_visits[sites_stat_visits['target']=='female']['site-id'].value_counts().nlargest(20)

site-id
3.0      13521
20.0      3713
16.0      2615
13.0      2515
29.0      1960
7.0       1959
34.0      1922
371.0     1542
65.0      1409
8.0       1383
62.0      1374
208.0     1279
98.0      1265
23.0      1225
46.0      1152
2.0       1143
21.0      1125
42.0      1059
41.0      1059
206.0      962
Name: count, dtype: int64

In [113]:
sites_stat_visits[sites_stat_visits['target']=='male']['site-id'].value_counts().nlargest(20)

site-id
3.0      13340
20.0      4459
16.0      3393
13.0      2609
29.0      2555
23.0      2433
2.0       1920
153.0     1674
41.0      1550
7.0       1434
21.0      1386
371.0     1321
206.0     1307
37.0      1182
34.0       897
147.0      885
62.0       838
98.0       828
33.0       801
307.0      795
Name: count, dtype: int64

# site-meta

In [87]:
val_site_meta = val_df_fixed[['user_id', 'target', 'site-meta']].copy()
val_site_meta

,user_id,target,site-meta
0,user_127756,female,[{'site-id': 407}]
1,user_127757,male,"[{'site-id': 16, 'recency': 1, 'frequency': 5,..."
2,user_127758,female,"[{'site-id': 391}, {'site-id': 42}, {'site-id'..."
3,user_127759,female,"[{'site-id': 93, 'recency': 5, 'frequency': 2,..."
4,user_127760,female,"[{'site-id': 46}, {'site-id': 29}, {'site-id':..."
...,...,...,...
27442,user_155198,female,"[{'site-id': 49, 'recency': 1, 'frequency': 1,..."
27443,user_155199,male,"[{'site-id': 21}, {'site-id': 17}]"
27444,user_155200,female,"[{'site-id': 34, 'recency': 1, 'frequency': 1,..."
27445,user_155201,male,"[{'site-id': 218, 'recency': 1, 'frequency': 1..."


In [88]:
levels_labels = ['site-meta']
val_site_meta_flat = explode_column(val_site_meta, levels_labels)
val_site_meta_flat

,user_id,target,site-id,recency,frequency,monetary
0,user_127756,female,407,NaN,NaN,NaN
1,user_127757,male,16,1.0,5.0,4.0
2,user_127757,male,29,NaN,NaN,NaN
3,user_127757,male,21,1.0,2.0,5.0
4,user_127757,male,40,1.0,1.0,5.0
...,...,...,...,...,...,...
302020,user_155202,female,76,NaN,NaN,NaN
302021,user_155202,female,3,NaN,NaN,NaN
302022,user_155202,female,603,NaN,NaN,NaN
302023,user_155202,female,86,1.0,1.0,3.0


# exchange-sessions

In [89]:
val_exch_sess = val_df_fixed[['user_id', 'target', 'exchange-sessions']].copy()
val_exch_sess

,user_id,target,exchange-sessions
0,user_127756,female,"[{'landed-at': 1697132982, 'sites': [169, 214,..."
1,user_127757,male,"[{'landed-at': 1693508972, 'sites': [2, 169, 7..."
2,user_127758,female,NaN
3,user_127759,female,"[{'landed-at': 1697791749, 'sites': [373, 169,..."
4,user_127760,female,NaN
...,...,...,...
27442,user_155198,female,"[{'landed-at': 1650900543, 'sites': [71, 326, ..."
27443,user_155199,male,NaN
27444,user_155200,female,NaN
27445,user_155201,male,NaN


In [90]:
levels_labels = ['exchange-sessions', 'clicks']
val_exch_sess_flat = explode_column(val_exch_sess, levels_labels)
val_exch_sess_flat

,user_id,target,landed-at,sites,accepted-site-id,accepted-at,clicked-at,site-id
0,user_127756,female,1.697133e+09,"[169, 214, 232, 244, 405, 229, 83, 231, 2, 21,...",262.0,1.697133e+09,NaN,NaN
1,user_127756,female,1.697126e+09,"[169, 214, 837, 476, 216, 215, 2, 83, 21, 244,...",169.0,1.697126e+09,NaN,NaN
2,user_127757,male,1.693509e+09,"[2, 169, 71, 236, 218, 21, 240, 215, 239, 226,...",42.0,1.693509e+09,1.693509e+09,42.0
3,user_127758,female,NaN,NaN,NaN,NaN,NaN,NaN
4,user_127759,female,1.697792e+09,"[373, 169, 214, 355, 71, 217, 2, 1, 21, 226, 8...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
177067,user_155202,female,1.670821e+09,"[21, 2, 355, 373, 274, 237, 364, 223, 236, 221...",NaN,NaN,1.670821e+09,169.0
177068,user_155202,female,1.666088e+09,"[246, 305, 221, 2, 71, 21, 480, 234, 28, 330, ...",NaN,NaN,NaN,NaN
177069,user_155202,female,1.608743e+09,"[2, 21, 339, 1, 50, 40, 173, 80, 89, 159, 119,...",175.0,1.608743e+09,1.608743e+09,175.0
177070,user_155202,female,1.549374e+09,"[49, 34, 86, 183, 179, 6, 182, 44, 96, 57, 40,...",NaN,NaN,NaN,NaN


# last-visits-in-categories

In [91]:
val_last_visits = val_df_fixed[['user_id', 'target', 'last-visits-in-categories']].copy()
val_last_visits

,user_id,target,last-visits-in-categories
0,user_127756,female,"[{'category': 'other', 'last-visit-at': 169713..."
1,user_127757,male,"[{'category': 'electronics', 'last-visit-at': ..."
2,user_127758,female,"[{'category': 'sport', 'last-visit-at': 165060..."
3,user_127759,female,"[{'category': 'furniture', 'last-visit-at': 16..."
4,user_127760,female,"[{'category': 'other', 'last-visit-at': 169765..."
...,...,...,...
27442,user_155198,female,"[{'category': 'other', 'last-visit-at': 169699..."
27443,user_155199,male,"[{'category': 'hypermarket', 'last-visit-at': ..."
27444,user_155200,female,"[{'category': 'furniture', 'last-visit-at': 16..."
27445,user_155201,male,"[{'category': 'other', 'last-visit-at': 169911..."


In [92]:
levels_labels = ['last-visits-in-categories']
val_last_visits_flat = explode_column(val_last_visits, levels_labels)
val_last_visits_flat

,user_id,target,category,last-visit-at
0,user_127756,female,other,1.697133e+09
1,user_127756,female,education,1.699115e+09
2,user_127757,male,electronics,1.698422e+09
3,user_127757,male,other,1.698423e+09
4,user_127757,male,shoes,1.698424e+09
...,...,...,...,...
156326,user_155201,male,education,1.695401e+09
156327,user_155201,male,luxury,1.697236e+09
156328,user_155201,male,entertainment_services,1.697235e+09
156329,user_155201,male,media,1.697718e+09


In [122]:
sites_last_visits = val_last_visits_flat[['user_id', 'target', 'category']]
sites_last_visits = sites_last_visits.drop_duplicates()
sites_last_visits

,user_id,target,category
0,user_127756,female,other
1,user_127756,female,education
2,user_127757,male,electronics
3,user_127757,male,other
4,user_127757,male,shoes
...,...,...,...
156326,user_155201,male,education
156327,user_155201,male,luxury
156328,user_155201,male,entertainment_services
156329,user_155201,male,media


In [127]:
top20_cat_female = sites_last_visits[sites_last_visits['target']=='female']['category'].value_counts().nlargest(20)
top20_cat_female

category
other                     13565
electronics                6466
hypermarket                6088
entertainment_services     5521
cosmetics                  5477
fashion                    5275
bank                       4595
media                      4300
food                       4055
furniture                  3899
education                  3539
health                     3296
kids                       2976
travel                     2710
luxury                     1969
pets                       1967
shoes                      1931
household_appliances       1601
gifts                      1448
insurance                  1232
Name: count, dtype: int64

In [126]:
top20_cat_male = sites_last_visits[sites_last_visits['target']=='male']['category'].value_counts().nlargest(20)
top20_cat_male

category
other                     13409
electronics                7463
hypermarket                6528
bank                       5900
entertainment_services     4488
media                      4406
fashion                    3612
food                       3595
cosmetics                  2790
furniture                  2727
education                  2592
household_appliances       2213
health                     2027
travel                     1980
insurance                  1554
kids                       1496
luxury                     1309
shoes                      1070
pets                        938
gifts                       764
Name: count, dtype: int64

In [130]:
set(top20_cat_female.index.tolist()) == set(top20_cat_male.index.tolist())

True